<a href="https://colab.research.google.com/github/bettyzhu912/2025-CAS-RPM-NN-GenAI-Workshop/blob/working-branch/CAS_RPM_Workshop_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Useful packages, installations, and imports for this exercise

In [4]:
from google.colab import userdata
import os
GITHUB_TOKEN = userdata.get('GITHUB_SECRET')

# os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN


In [8]:
! git clone --branch working-branch https://{GITHUB_TOKEN}@github.com/bettyzhu912/2025-CAS-RPM-NN-GenAI-Workshop.git

Cloning into '2025-CAS-RPM-NN-GenAI-Workshop'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 29 (delta 0), reused 1 (delta 0), pack-reused 25 (from 3)
Receiving objects: 100% (29/29), 42.57 MiB | 25.57 MiB/s, done.


In [10]:
# Run This Cell!
# This just imports some packages which we will be using for this exercise

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import linear_model

from scipy import sparse

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

from gensim.corpora.dictionary import Dictionary
from collections import Counter, defaultdict

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# The Dataset

In [12]:
df = pd.read_csv('/content/2025-CAS-RPM-NN-GenAI-Workshop/prompt.csv')

In [13]:
df.head(10)

,personality,prompt,response,response2,sus,cert,fraud_prob,repudiation
0,You are a claims adjuster at a California home...,You just received a call about a wildfire. The...,The policyholder reported damage due to a near...,Further investigation is necessary to assess t...,average,high,0.433,False
1,You are a claims adjuster at a California home...,You just received a call about a wildfire. nea...,The policyholder reported potential fire damag...,The policyholder indicated that they have disc...,high,high,0.651,False
2,You are a claims adjuster at a California home...,You just received a call about a wildfire. The...,The policyholder reported damage from a recent...,Further investigation is required to assess th...,high,high,0.721,False
3,You are a claims adjuster at a California home...,You just received a call about a wildfire. The...,The policyholder reported damage from the rece...,The homeowner confirmed that the charred music...,average,high,0.452,False
4,You are a claims adjuster at a California home...,You just received a call about a wildfire. The...,The claimant reported damage to their property...,The claim will require further investigation t...,average,high,0.504,False
5,You are a claims adjuster at a California home...,You just received a call about a wildfire. The...,The policyholder reported damage from a recent...,Further investigation is required to fully ass...,high,average,0.627,False
6,You are a claims adjuster at a California home...,You just received a call about a wildfire. The...,The policyholder reported a wildfire incident ...,The policyholder confirmed that there are no s...,average,high,0.456,False
7,You are a claims adjuster at a California home...,You just received a call about a wildfire. The...,Claimant reported damage from a recent wildfir...,After a thorough investigation into the wildfi...,average,high,0.568,False
8,You are a claims adjuster at a California home...,You just received a call about a wildfire. The...,The claimant reported damage to the main struc...,The claimant reiterated the engineer's recomme...,average,high,0.516,False
9,You are a claims adjuster at a California home...,You just received a call about a wildfire. The...,The policyholder reported a wildfire event tha...,"During the follow-up, I encountered a delay in...",high,average,0.499,False


In [23]:
df['response'].value_counts()

,count
response,
"The policyholder reported damage due to a nearby wildfire, affecting both the main structure and personal property. Notably, there is moderate staining observed on the ceiling and upper walls from smoke exposure, which will require assessment and potential remediation. Additionally, the insured mentioned incurring storage unit fees for salvaged personal belongings, which could be relevant for further investigation into recovery and reimbursement options. Claim development will focus on these aspects to ensure a comprehensive evaluation of damages and associated costs.",1
"The claimant reported damage to the exterior of their home following an incident involving burning pine cones that popped and scattered embers, igniting nearby vegetation. The policyholder has a coverage limit of $261,000 for the main structure and $121,000 for contents, which may be relevant if further damages are identified. Additional details regarding the extent of the fire and any potential impact on nearby structures remain unclear at this stage and will require further investigation.",1
"The insured reported an incident involving an overheating issue caused by a malfunctioning dryer door switch, which resulted in damage to the laundry room floor and surrounding areas. The insured confirmed they have coverage for both the main structure and contents, and they are currently assessing the extent of the damage. Additional information on repairs and any personal property affected will be collected as the claim progresses.",1
"The policyholder reported an external fire incident involving a brush pile in the backyard that spontaneously combusted, resulting in damage to the surrounding area. They have coverage for both the main structure and contents, and while the full extent of the damages is still being assessed, initial observations indicate potential impacts on nearby landscaping and structures. Further investigation will be required to determine the full scope of the claim.",1
"Claimant reported an internal fire incident in their garage, caused by an E-bike battery igniting, resulting in smoke damage throughout the main structure. The homeowner's total coverage for the dwelling is $383,000, with contents coverage at $184,000, indicating potential extensive damage to both property and personal belongings. Further investigation will be necessary to assess the full extent of the damage and determine the appropriate response for the claim. Claimant will provide additional details regarding any affected personal items once the fire department completes their report.",1
...,...
"The claimant reported an internal fire incident in their home in Huntington Park, caused by an e-cig battery explosion on the coffee table, resulting in damage to surrounding furniture and potential smoke damage throughout the living area. The policyholder mentioned that while the fire was quickly contained, they are concerned about the extent of the damage and whether their contents coverage will adequately address the losses. Further investigation will be necessary to assess the full impact on both the structure and personal property, given the total policy limits of $311,000 for the main structure and $203,000 for contents. The claimant will provide additional details as they gather information on damaged items and any related expenses.",1
"The claimant reported an external fire incident involving their property, where it appears that kids were playing with firecrackers near the carport, which may have contributed to the blaze. The policyholder has coverage for the main structure and contents, and they noted that the fire caused damage to both areas. Further investigation is needed to determine the extent of the damage and any potential liability issues related to the activities of the children. The claimant mentioned that they had recently used their outdoor fireplace, but did not specify whether the chimney was clogged with ash at the time of the incident. Additi

In [24]:

# Based on Info159 - Natural Language Processing HW 1 by Prof. David Bamman
class FeaturizedDataLoader:
    def __init__(self,
                 csv_file,
                 feature_method,
                 test_size=0.2,
                 random_state=42,
                 min_feature_count=1):
        """
        Args:
            csv_file: Path to your CSV file
            feature_method: A function that takes in text and returns a dictionary of features
            test_size: Fraction of data to hold out for test
            random_state: Random seed for reproducible splits
            min_feature_count: Minimum frequency of a feature for it to be included in the vocabulary
        """
        self.feature_vocab = {}
        self.feature_method = feature_method
        self.min_feature_count = min_feature_count

        # 1) Load the data from the CSV
        data = self.load_data(csv_file)

        # 2) Split into train and test
        train_data, test_data = train_test_split(data,
                                                 test_size=test_size,
                                                 random_state=random_state)

        # 3) Process the data
        self.trainX, self.trainY, self.trainOrig = self.process(train_data, training=True)
        self.testX, self.testY, self.testOrig   = self.process(test_data, training=False)

    def load_data(self, csv_file):
        """
        Loads the CSV and produces a list of tuples: (idd, label, text),
        where label = 'repudiation' and text = concatenation of 'response' and 'response2'.
        """
        df = pd.read_csv(csv_file)
        data = []

        # You can decide how you want to define `idd`; here, we'll just use the row index.
        for idx, row in df.iterrows():
            idd = idx
            label = row['repudiation']

            # Concatenate response + response2 (handle NaN or missing values as needed)
            # Use str() to ensure concatenation works even if they are numeric or missing.
            response_text = str(row['response']) if pd.notnull(row['response']) else ""
            response2_text = str(row['response2']) if pd.notnull(row['response2']) else ""
            text = response_text + " " + response2_text

            data.append((idd, label, text))
        return data

    def featurize(self, data):
        """
        Featurizes the data using the user-defined feature_method.
        """
        featurized_data = []
        for idd, label, text in data:
            feats = self.feature_method(text)
            featurized_data.append((label, feats))
        return featurized_data

    def process(self, data, training=False):
        """
        Converts data into a sparse matrix (dok_matrix) X and label array Y.
        If training=True, also builds/updates the feature vocabulary.
        """
        # data is a list of (idd, label, text)
        # We only need: label, text to featurize.
        # Keep original_data for reference or debugging
        original_data = data

        # 1) Featurize
        featurized_data = self.featurize(data)  # -> list of (label, {feat: count, ...})

        # 2) Build vocab if training
        if training:
            fid = 0
            feature_doc_count = Counter()
            for label, feats in featurized_data:
                for feat in feats:
                    feature_doc_count[feat] += 1

            for feat, count in feature_doc_count.items():
                if count >= self.min_feature_count:
                    self.feature_vocab[feat] = fid
                    fid += 1

        # 3) Create the sparse matrix X, label array Y
        F = len(self.feature_vocab)
        D = len(featurized_data)

        X = sparse.dok_matrix((D, F), dtype=float)
        Y = [None]*D

        for idx, (label, feats) in enumerate(featurized_data):
            for feat, val in feats.items():
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = val
            Y[idx] = label

        return X, Y, original_data

    def load_test(self, dataFile):
        """
        If you need a separate method to load an external test CSV in the future,
        similar to your old code, you could adapt it here.
        This is just a placeholder to show how it might be adapted.
        """
        # You can re-use the same logic:
        # 1) load new data (with the same columns)
        # 2) featurize
        # 3) create X
        # 4) but you won't update self.feature_vocab
        pass


# Building a Simple Classifier

## Encoding the Input

### Bag of Words

In [25]:
def bag_of_words(text):
    # Here the `feats` dict should contain the features -- the key should be the feature name,
    # and the value is the feature value.  See `simple_featurize` for an example.

    feats = {}
    words = nltk.word_tokenize(text)

    for word in words:
      word=word.lower()

      feats[word] = 1
    return feats


In [26]:
BoW_Loader = FeaturizedDataLoader('/content/2025-CAS-RPM-NN-GenAI-Workshop/prompt.csv', bag_of_words)

In [27]:
# let's take a look

# dimensions
D, F = BoW_Loader.trainX.shape
print("number of rows in train X:", D)
print("number of features:", F)

number of rows in train X: 4605
number of features: 4799


### Embeddings

In [28]:
# Fetch embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2025-03-01 06:46:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-03-01 06:46:23--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-03-01 06:46:23--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [29]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [30]:
# To make use of these embeddings, we need to ensure we have a mapping of words to indices
# Let's define a simple word based tokenizer to work with these embeddings

class WordTokenizer:
    def __init__(self, documents=None, vocab_size=100000):
        if documents:
          self.id2token = Dictionary(
              [nltk.word_tokenize(document.lower()) for document in documents],
              prune_at=vocab_size
          )
          self.id2token.filter_extremes(no_below=1, no_above=1, keep_n=vocab_size)
        else:
          self.id2token = Dictionary(prune_at = vocab_size)
        self.vocab_size = vocab_size

    def __call__(self, text):
        return self.tokenize(text)

    def tokenize(self, text):
        tokens = nltk.word_tokenize(text.lower())
        return self.id2token.doc2idx(tokens, unknown_word_index = self.get_vocab_size())

    def get_vocab(self):
        return self.id2token.token2id

    def get_vocab_size(self):
        return len(self.get_vocab())

    def get_word_id(self, word):
        return self.get_vocab()[word]

    def get_word(self, idx):
        return self.id2token.get(idx, None)

    def add_documents(self, documents):
        self.id2token.add_documents([nltk.word_tokenize(document.lower()) for document in documents], prune_at=self.vocab_size)
        self.id2token.filter_extremes(no_below=1, no_above=1, keep_n=self.vocab_size)

In [31]:
# Now let's build an embedding matrix based on this tokenizer and the embeddings we loaded earlier

def create_embedding_matrix(tokenizer: WordTokenizer, embeddings_index, embedding_dim=100):
    vocab_size = tokenizer.get_vocab_size() + 1  # +1 for OOV token if needed
    embedding_matrix = np.zeros((vocab_size, embedding_dim)).astype('float32')

    for word in tokenizer.get_vocab():
        if word in embeddings_index:
            idx = tokenizer.get_word_id(word)
            embedding_matrix[idx] = embeddings_index[word]

    return embedding_matrix



In [32]:
# recall our dataloader stores the original training data in the trainOrig attribute as a list of (idd, label, text)

# now we extract the text from this

trainOrigDocuments = [x[2] for x in BoW_Loader.trainOrig]

word_tokenizer = WordTokenizer(trainOrigDocuments, 10000)
embedding_matrix = create_embedding_matrix(word_tokenizer, embeddings_index)

In [33]:
embedding_matrix.shape

(4800, 100)

In [43]:
(embedding_matrix)

array([[-0.10767 ,  0.11053 ,  0.59812 , ..., -0.83155 ,  0.45293 ,
         0.082577],
       [-0.33979 ,  0.20941 ,  0.46348 , ..., -0.23394 ,  0.47298 ,
        -0.028803],
       [-0.27086 ,  0.044006, -0.02026 , ..., -0.4923  ,  0.63687 ,
         0.23642 ],
       ...,
       [-0.26265 , -0.45791 ,  0.31674 , ...,  0.018427,  0.025385,
         0.45626 ],
       [-0.14031 , -0.45892 , -0.14656 , ..., -0.04242 ,  0.73645 ,
         0.54508 ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [44]:
word_tokenizer.get_word(0)

','

In [45]:
embeddings_index['!']

array([ 0.38472  ,  0.49351  ,  0.49096  , -1.5434   , -0.33614  ,
        0.6222   ,  0.32265  ,  0.075331 ,  0.65591  , -0.23517  ,
        1.2114   ,  0.06193  , -0.62004  ,  0.31371  ,  0.38948  ,
       -0.24381  , -0.065643 ,  0.58797  , -0.86382  ,  0.63166  ,
        0.68363  ,  0.39647  , -0.62388  , -0.25094  ,  0.92831  ,
        1.5152   , -0.43917  ,  0.22249  ,  1.3695   , -0.53098  ,
        0.39811  ,  0.77114  ,  0.49043  ,  0.58853  ,  0.2376   ,
        0.3162   , -0.011962 , -0.047074 ,  0.34585  , -1.2944   ,
        0.18597  ,  0.27002  , -0.70602  , -0.20652  , -0.25194  ,
       -0.4868   , -0.71538  , -0.23887  , -0.041612 , -0.55488  ,
       -0.54226  ,  0.21236  ,  0.025341 ,  0.96517  , -0.88183  ,
       -1.8681   ,  0.32657  ,  1.1689   ,  1.1759   , -0.17393  ,
       -0.3371   ,  0.87535  , -1.0114   , -0.6181   ,  1.008    ,
        0.31506  ,  0.24417  ,  0.064393 ,  0.33678  ,  0.33632  ,
        0.45975  ,  0.22813  , -0.37505  , -0.37508  ,  0.0893

## Defining a Model

### Linear Regression

In [37]:
L2_regularization_strength = 1.0

In [39]:
# Train a Logistic Regression https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
log_reg = LogisticRegression(C = L2_regularization_strength, max_iter=1000)
log_reg.fit(BoW_Loader.trainX, BoW_Loader.trainY)

# score it against our data
training_accuracy = log_reg.score(BoW_Loader.trainX, BoW_Loader.trainY)
validation_accuracy = log_reg.score(BoW_Loader.testX, BoW_Loader.testY)

print("Method: %s, Features: %s, Train accuracy: %.3f, validation accuracy: %.3f" % (BoW_Loader.feature_method.__name__, F, training_accuracy, validation_accuracy))

Method: bag_of_words, Features: 4799, Train accuracy: 0.997, Dev accuracy: 0.974


In [55]:
# let's get a grasp of the distribution to frame these results

print("Train Positive Proportion:", sum(BoW_Loader.trainY)/len(BoW_Loader.trainY))
print("Train Negative Proportion:", 1 - sum(BoW_Loader.trainY)/len(BoW_Loader.trainY))
print("Train Positive Count:", sum(BoW_Loader.trainY))
print("Train Count:", len(BoW_Loader.trainY))
print("Test Positive Proportion:", sum(BoW_Loader.testY)/len(BoW_Loader.testY))
print("Test Negative Proportion:", 1 - sum(BoW_Loader.testY)/len(BoW_Loader.testY))
print("Test Positive Count:", sum(BoW_Loader.testY))
print("Test Count:", len(BoW_Loader.testY))

Train Positive Proportion: 0.02975027144408252
Train Negative Proportion: 0.9702497285559175
Train Positive Count: 137
Train Count: 4605
Test Positive Proportion: 0.019097222222222224
Test Negative Proportion: 0.9809027777777778
Test Positive Count: 22
Test Count: 1152


### Multilayer Perceptron

In [56]:
# Create a Torch Dataset and DataLoader from our data

class BoW_Dataset(Dataset):
    def __init__(self, X, Y):
        self.X = X.toarray().astype(np.float32)
        self.Y = Y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]

        # specific to our data
        if y == True:
          y = 1
        else:
          y = 0
        return x, np.array([y]).astype(np.float32)

train_dataset = BoW_Dataset(BoW_Loader.trainX, BoW_Loader.trainY)
valid_dataset = BoW_Dataset(BoW_Loader.testX, BoW_Loader.testY)

# dataloader = DataLoader(train_dataset, batch_size=2)

In [68]:
model = nn.Sequential(
    nn.Linear(F, 1),
    # nn.Sigmoid()
)
print(model)

Sequential(
  (0): Linear(in_features=4799, out_features=1, bias=True)
  (1): Sigmoid()
)


In [77]:
# TODO figure out default hyperparams

def run_training_loop(model, batch_size=32, n_epochs=10, lr=1e-3, weight_decay = 1e-4):

    # We could write our training procedure manually and directly index the `Dataset` objects,
    # but the `DataLoader` object conveniently creates an iterable for automatically creating random minibatches:
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

    # Choose Adam as the optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay = weight_decay)

    # Use the cross entropy loss function
    # loss_fn = nn.CrossEntropyLoss()
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([97/3]))
    # store metrics
    train_loss_history = np.zeros([n_epochs, 1])
    valid_accuracy_history = np.zeros([n_epochs, 1])
    valid_loss_history = np.zeros([n_epochs, 1])

    for epoch in range(n_epochs):

        # Some layers, such as Dropout, behave differently during training
        model.train()

        train_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            # Erase accumulated gradients
            optimizer.zero_grad()

            # Forward pass
            output = model(data)

            # Calculate loss
            loss = loss_fn(output, target)
            train_loss += loss.item()

            # Backward pass
            loss.backward()

            # Weight update
            optimizer.step()

        train_loss_history[epoch] = train_loss / len(train_loader.dataset)

        # Track loss each epoch
        print('Train Epoch: %d  Average loss: %.4f' %
              (epoch + 1,  train_loss_history[epoch]))

        # Putting layers like Dropout into evaluation mode
        model.eval()

        valid_loss = 0
        correct = 0

        # Turning off automatic differentiation
        with torch.no_grad():
            for data, target in valid_loader:
                output = model(data)
                valid_loss += loss_fn(output, target).item()  # Sum up batch loss
                pred = torch.round(output)
                # pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
                correct += pred.eq(target.view_as(pred)).sum().item()

        valid_loss_history[epoch] = valid_loss / len(valid_loader.dataset)
        valid_accuracy_history[epoch] = correct / len(valid_loader.dataset)

        print('Valid set: Average loss: %.4f, Accuracy: %d/%d (%.4f)\n' %
              (valid_loss_history[epoch], correct, len(valid_loader.dataset),
              100. * valid_accuracy_history[epoch]))

    return model, train_loss_history, valid_loss_history, valid_accuracy_history

In [80]:
logistic_nn, train_loss_history, valid_loss_history, valid_accuracy_history = run_training_loop(model, batch_size=32, n_epochs=10, lr=1e-2, weight_decay = 0)

<ipython-input-77-0e33d6c9b6be>:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Train Epoch: %d  Average loss: %.4f' %
<ipython-input-77-0e33d6c9b6be>:66: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Valid set: Average loss: %.4f, Accuracy: %d/%d (%.4f)\n' %


Train Epoch: 1  Average loss: 0.0307
Valid set: Average loss: 0.0356, Accuracy: 1076/1152 (93.4028)

Train Epoch: 2  Average loss: 0.0306
Valid set: Average loss: 0.0351, Accuracy: 1099/1152 (95.3993)

Train Epoch: 3  Average loss: 0.0305
Valid set: Average loss: 0.0351, Accuracy: 1101/1152 (95.5729)

Train Epoch: 4  Average loss: 0.0305
Valid set: Average loss: 0.0350, Accuracy: 1111/1152 (96.4410)

Train Epoch: 5  Average loss: 0.0305
Valid set: Average loss: 0.0351, Accuracy: 1101/1152 (95.5729)

Train Epoch: 6  Average loss: 0.0305
Valid set: Average loss: 0.0351, Accuracy: 1098/1152 (95.3125)

Train Epoch: 7  Average loss: 0.0305
Valid set: Average loss: 0.0351, Accuracy: 1097/1152 (95.2257)

Train Epoch: 8  Average loss: 0.0305
Valid set: Average loss: 0.0350, Accuracy: 1106/1152 (96.0069)

Train Epoch: 9  Average loss: 0.0305
Valid set: Average loss: 0.0350, Accuracy: 1106/1152 (96.0069)

Train Epoch: 10  Average loss: 0.0305
Valid set: Average loss: 0.0350, Accuracy: 1109/1152 

In [51]:
model = nn.Sequential(
    nn.Linear(F, 1024),
    nn.ReLU(),
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Linear(256, 1),
    nn.Sigmoid()
)
print(model)

Sequential(
  (0): Linear(in_features=4799, out_features=1024, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1024, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=1, bias=True)
  (5): Sigmoid()
)


In [52]:
len(word_tokenizer.get_vocab())
word_tokenizer.vocab_size

10000

In [53]:
mlp, train_loss_history, valid_loss_history, valid_accuracy_history = run_training_loop(model, batch_size=32, n_epochs=5, lr=1e-3)

<ipython-input-49-031e17bf88a2>:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Train Epoch: %d  Average loss: %.4f' %


Train Epoch: 1  Average loss: 0.0004


<ipython-input-49-031e17bf88a2>:66: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Valid set: Average loss: %.4f, Accuracy: %d/%d (%.4f)\n' %


Valid set: Average loss: 0.0000, Accuracy: 1152/1152 (100.0000)

Train Epoch: 2  Average loss: 0.0000
Valid set: Average loss: 0.0000, Accuracy: 1152/1152 (100.0000)

Train Epoch: 3  Average loss: 0.0000
Valid set: Average loss: 0.0000, Accuracy: 1152/1152 (100.0000)

Train Epoch: 4  Average loss: 0.0000
Valid set: Average loss: 0.0000, Accuracy: 1152/1152 (100.0000)

Train Epoch: 5  Average loss: 0.0000
Valid set: Average loss: 0.0000, Accuracy: 1152/1152 (100.0000)

Train Epoch: 6  Average loss: 0.0000
Valid set: Average loss: 0.0000, Accuracy: 1152/1152 (100.0000)

Train Epoch: 7  Average loss: 0.0000
Valid set: Average loss: 0.0000, Accuracy: 1152/1152 (100.0000)

Train Epoch: 8  Average loss: 0.0000
Valid set: Average loss: 0.0000, Accuracy: 1152/1152 (100.0000)

Train Epoch: 9  Average loss: 0.0000
Valid set: Average loss: 0.0000, Accuracy: 1152/1152 (100.0000)

Train Epoch: 10  Average loss: 0.0000
Valid set: Average loss: 0.0000, Accuracy: 1152/1152 (100.0000)



### Convolutional Neural Network

In [ ]:
# Let's define a dataset using our word tokenizer


class Tokenized_Dataset(Dataset):
    def __init__(self, X, Y, tokenizer, max_seq_len=500):
        self.X = X
        self.Y = Y
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]

        # use the tokenizer

        x = self.tokenizer(x)

        # Pad or truncate to max_seq_len
        if len(x) < self.max_seq_len:
            x = x + [self.tokenizer.get_vocab_size()] * (self.max_seq_len - len(x))
        else:
            x = x[:self.max_seq_len]
        x = np.array(x)
        # specific to our data
        if y == True:
          y = 1
        else:
          y = 0
        return x, np.array([y]).astype(np.float32)

trainOrigDocuments = [x[2] for x in BoW_Loader.trainOrig]
validOrigDocuments = [x[2] for x in BoW_Loader.testOrig]

train_dataset = Tokenized_Dataset(trainOrigDocuments, BoW_Loader.trainY, word_tokenizer, max_seq_len = 500)
valid_dataset = Tokenized_Dataset(validOrigDocuments, BoW_Loader.testY, word_tokenizer, max_seq_len = 500)

In [ ]:
# let's try to use the embeddings we loaded earlier now
model = nn.Sequential(
    nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix)) # by default this is frozen ie will not be updated during training
)
print(model)

Sequential(
  (0): Embedding(10001, 100)
)


In [ ]:
# let's mimic what will happen with our embeddings
max_seq_len = 500

x = BoW_Loader.trainOrig[0][2]
x = word_tokenizer(x)
padding_token = word_tokenizer.get_vocab_size()

# Pad or truncate to max_seq_len
if len(x) < max_seq_len:
    x = x + [word_tokenizer.get_vocab_size()] * (max_seq_len - len(x))
else:
    x = x[:max_seq_len]
print("padding and unknown token is:", padding_token)
print("Tokenized Sequence is:")
print(np.array(x))

padding and unknown token is: 10000
Tokenized Sequence is:
[   50    94     4    75     1   126   156   130    89   105     8     6
     3   133    19    63     4    52   153    68   130   127    78    16
   110   129    19   113   129   155    55    90   152    88    26    17
     9     7     3   130    84    69    92    51    13   101     4   122
    66   148   155    55   118    16   126   159    42   130    85   140
    11    69    67    25   130    84    69   147   111     5    68    15
    96   141   131     2    77   130    98    64   149   139     3     4
   160    30    61   130   116    57   137   130   108    21   138     5
    64    80   129     5   130    84    40    92    71   160    27   156
    86     4    91    45     8    70    76   160   117   144   150    69
    58    69    10    74    93   119   129    54    15    22   129    69
    49    15   145    46     5   130   120    72    93    22    53    29
   125    73    68    47   154   121     5    92    51    13   10

In [ ]:
model(torch.tensor(np.array([x])))

tensor([[[-0.0201,  0.0375,  0.3536,  ...,  0.0626,  0.2839, -0.3163],
         [-0.2946, -0.2074,  0.4911,  ...,  0.5775,  0.5217, -0.0270],
         [-0.1077,  0.1105,  0.5981,  ..., -0.8316,  0.4529,  0.0826],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])

In [ ]:
model(torch.tensor(np.array([x]))).shape

torch.Size([1, 500, 100])

In [ ]:
# notice that this is currently (1, seq_len, embedding_dim) when we use this in our dataloader we'll have (batch_size, seq_len, embedding_dim) but the convolutional layers we use expect (batch_size, embedding_dim, seq_len)

# Let's look at how we can create a custom layer for use in Pytorch that will change this for us

class Transpose(nn.Module):
    def __init__(self, dim0, dim1):
        super().__init__()
        self.dim0 = dim0
        self.dim1 = dim1

    def forward(self, x):
        x = torch.transpose(x,self.dim0, self.dim1)
        return x

In [ ]:
# a utility to help us calculate the dims of the hidden layer following our convolution layer

def calc_linear_dim(max_seq_len, num_filters, kernel_size, stride, max_pool_size):
  """
  Calculates the in size of a linear layer following convolutions and max pooling

  Assumes no padding is used
  """
  return ((max_seq_len-kernel_size)//stride + 1)//max_pool_size*num_filters

In [ ]:
# We'll use this in combination with a CNN since now we have sequential data
# let's try to use the embeddings we loaded earlier now

# recall max_seq_len is 500 right now

max_seq_len = 500
num_filters = 64
kernel_size = 5
stride = 1
max_pool_size = 20

linear_size = calc_linear_dim(max_seq_len, num_filters, kernel_size, stride, max_pool_size)

model = nn.Sequential(
    nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix)), # by default this is frozen ie will not be updated during training
    Transpose(1, 2),
    nn.Conv1d(100, num_filters, kernel_size),
    nn.ReLU(),
    nn.MaxPool1d(max_pool_size),
    nn.Flatten(),
    nn.Linear(linear_size, 1),
    nn.Sigmoid()
)
print(model)


Sequential(
  (0): Embedding(10001, 100)
  (1): Transpose()
  (2): Conv1d(100, 64, kernel_size=(5,), stride=(1,))
  (3): ReLU()
  (4): MaxPool1d(kernel_size=20, stride=20, padding=0, dilation=1, ceil_mode=False)
  (5): Flatten(start_dim=1, end_dim=-1)
  (6): Linear(in_features=1536, out_features=1, bias=True)
  (7): Sigmoid()
)


In [ ]:
cnn, train_loss_history, valid_loss_history, valid_accuracy_history = run_training_loop(model, batch_size=32, n_epochs=15, lr=5e-3)

<ipython-input-21-031e17bf88a2>:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Train Epoch: %d  Average loss: %.4f' %


Train Epoch: 1  Average loss: 0.0230


<ipython-input-21-031e17bf88a2>:66: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Valid set: Average loss: %.4f, Accuracy: %d/%d (%.4f)\n' %


Valid set: Average loss: 0.0222, Accuracy: 504/1000 (50.4000)

Train Epoch: 2  Average loss: 0.0219
Valid set: Average loss: 0.0220, Accuracy: 545/1000 (54.5000)

Train Epoch: 3  Average loss: 0.0208
Valid set: Average loss: 0.0211, Accuracy: 610/1000 (61.0000)

Train Epoch: 4  Average loss: 0.0195
Valid set: Average loss: 0.0201, Accuracy: 663/1000 (66.3000)

Train Epoch: 5  Average loss: 0.0166
Valid set: Average loss: 0.0188, Accuracy: 684/1000 (68.4000)

Train Epoch: 6  Average loss: 0.0134
Valid set: Average loss: 0.0185, Accuracy: 701/1000 (70.1000)

Train Epoch: 7  Average loss: 0.0096
Valid set: Average loss: 0.0195, Accuracy: 715/1000 (71.5000)

Train Epoch: 8  Average loss: 0.0063
Valid set: Average loss: 0.0238, Accuracy: 700/1000 (70.0000)

Train Epoch: 9  Average loss: 0.0032
Valid set: Average loss: 0.0211, Accuracy: 713/1000 (71.3000)

Train Epoch: 10  Average loss: 0.0017
Valid set: Average loss: 0.0222, Accuracy: 719/1000 (71.9000)

Train Epoch: 11  Average loss: 0.000

In [ ]:
# recall max_seq_len is 500 right now
num_filters = 64
kernel_size = 5
stride = 1
max_pool_size = 20

linear_size = calc_linear_dim(max_seq_len, num_filters, kernel_size, stride, max_pool_size)

model = nn.Sequential(
    nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1]), # let's fit our own custom embeddings on this data of the same shape
    Transpose(1, 2),
    nn.Conv1d(embedding_matrix.shape[1], num_filters, kernel_size),
    nn.ReLU(),
    nn.MaxPool1d(max_pool_size),
    nn.Flatten(),
    nn.Linear(linear_size, 1),
    nn.Sigmoid()
)
print(model)


Sequential(
  (0): Embedding(10001, 100)
  (1): Transpose()
  (2): Conv1d(100, 64, kernel_size=(5,), stride=(1,))
  (3): ReLU()
  (4): MaxPool1d(kernel_size=20, stride=20, padding=0, dilation=1, ceil_mode=False)
  (5): Flatten(start_dim=1, end_dim=-1)
  (6): Linear(in_features=1536, out_features=1, bias=True)
  (7): Sigmoid()
)


In [ ]:
cnn, train_loss_history, valid_loss_history, valid_accuracy_history = run_training_loop(model, batch_size=32, n_epochs=15, lr=5e-3)

<ipython-input-21-031e17bf88a2>:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Train Epoch: %d  Average loss: %.4f' %


Train Epoch: 1  Average loss: 0.0324


<ipython-input-21-031e17bf88a2>:66: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Valid set: Average loss: %.4f, Accuracy: %d/%d (%.4f)\n' %


Valid set: Average loss: 0.0225, Accuracy: 541/1000 (54.1000)

Train Epoch: 2  Average loss: 0.0172
Valid set: Average loss: 0.0232, Accuracy: 532/1000 (53.2000)

Train Epoch: 3  Average loss: 0.0080
Valid set: Average loss: 0.0236, Accuracy: 556/1000 (55.6000)

Train Epoch: 4  Average loss: 0.0018
Valid set: Average loss: 0.0250, Accuracy: 554/1000 (55.4000)

Train Epoch: 5  Average loss: 0.0005
Valid set: Average loss: 0.0258, Accuracy: 548/1000 (54.8000)

Train Epoch: 6  Average loss: 0.0002
Valid set: Average loss: 0.0259, Accuracy: 558/1000 (55.8000)

Train Epoch: 7  Average loss: 0.0001
Valid set: Average loss: 0.0266, Accuracy: 551/1000 (55.1000)

Train Epoch: 8  Average loss: 0.0001
Valid set: Average loss: 0.0272, Accuracy: 554/1000 (55.4000)

Train Epoch: 9  Average loss: 0.0001
Valid set: Average loss: 0.0271, Accuracy: 549/1000 (54.9000)

Train Epoch: 10  Average loss: 0.0001
Valid set: Average loss: 0.0280, Accuracy: 547/1000 (54.7000)

Train Epoch: 11  Average loss: 0.000

In [ ]:
# recall max_seq_len is 500 right now
num_filters = 8
kernel_size = 5
stride = 1
max_pool_size = 20
embedding_dim = 32


linear_size = calc_linear_dim(max_seq_len, num_filters, kernel_size, stride, max_pool_size)

model = nn.Sequential(
    nn.Embedding(embedding_matrix.shape[0], embedding_dim), # let's fit our own custom embeddings with our choice of dimensions
    Transpose(1, 2),
    nn.Conv1d(embedding_dim, num_filters, kernel_size),
    nn.ReLU(),
    nn.MaxPool1d(max_pool_size),
    nn.Flatten(),
    nn.Linear(linear_size, 1),
    nn.Sigmoid()
)
print(model)

Sequential(
  (0): Embedding(10001, 32)
  (1): Transpose()
  (2): Conv1d(32, 8, kernel_size=(5,), stride=(1,))
  (3): ReLU()
  (4): MaxPool1d(kernel_size=20, stride=20, padding=0, dilation=1, ceil_mode=False)
  (5): Flatten(start_dim=1, end_dim=-1)
  (6): Linear(in_features=192, out_features=1, bias=True)
  (7): Sigmoid()
)


In [ ]:
cnn, train_loss_history, valid_loss_history, valid_accuracy_history = run_training_loop(model, batch_size=32, n_epochs=15, lr=5e-3)

<ipython-input-21-031e17bf88a2>:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Train Epoch: %d  Average loss: %.4f' %


Train Epoch: 1  Average loss: 0.0226


<ipython-input-21-031e17bf88a2>:66: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Valid set: Average loss: %.4f, Accuracy: %d/%d (%.4f)\n' %


Valid set: Average loss: 0.0222, Accuracy: 519/1000 (51.9000)

Train Epoch: 2  Average loss: 0.0202
Valid set: Average loss: 0.0224, Accuracy: 529/1000 (52.9000)

Train Epoch: 3  Average loss: 0.0167
Valid set: Average loss: 0.0220, Accuracy: 553/1000 (55.3000)

Train Epoch: 4  Average loss: 0.0114
Valid set: Average loss: 0.0233, Accuracy: 568/1000 (56.8000)

Train Epoch: 5  Average loss: 0.0057
Valid set: Average loss: 0.0239, Accuracy: 580/1000 (58.0000)

Train Epoch: 6  Average loss: 0.0022
Valid set: Average loss: 0.0249, Accuracy: 573/1000 (57.3000)

Train Epoch: 7  Average loss: 0.0010
Valid set: Average loss: 0.0260, Accuracy: 579/1000 (57.9000)

Train Epoch: 8  Average loss: 0.0006
Valid set: Average loss: 0.0266, Accuracy: 573/1000 (57.3000)

Train Epoch: 9  Average loss: 0.0004
Valid set: Average loss: 0.0273, Accuracy: 580/1000 (58.0000)

Train Epoch: 10  Average loss: 0.0003
Valid set: Average loss: 0.0281, Accuracy: 576/1000 (57.6000)

Train Epoch: 11  Average loss: 0.000